### Introdução

Delta Lake oferece uma camada de armazenamento que traz ACID transactions para Apache Spark e big data workloads. Esse notebook explora como realizar operações de merge, também conhecidas como upserts, que são essenciais para manter os dados atualizados sem duplicidade, usando Delta Lake no Databricks. Este exemplo prático abrange desde a leitura de dados existentes até a atualização e inserção de novos registros, culminando na remoção segura dos dados armazenados.

### Importando Bibliotecas

In [ ]:
from delta.tables import *

Importação da biblioteca necessária para trabalhar com tabelas Delta, que permite realizar operações transacionais, como merge, update e delete em conjuntos de dados.

### Leitura do DataFrame Original

In [ ]:
# Lendo o DataFrame original do Delta Lake
despachantes_df = spark.table("despachantes")
despachantes_df.show()

Carrega o DataFrame despachantes diretamente de uma tabela Spark SQL existente e exibe suas primeiras linhas. Esta é uma forma de verificar rapidamente os dados antes de realizar operações mais complexas.

### Preparação dos Dados para Inserção/Atualização

In [ ]:
# Criando um novo DataFrame com os dados que serão inseridos ou atualizados
novo_despachante = spark.createDataFrame([(1, "João", "Ativo", "São Paulo", 10000, "2023-07-05"),
                                          (11, "Maria", "Inativo", "Rio de Janeiro", 5000, "2023-07-05")],
                                         ["id", "nome", "status", "cidade", "vendas", "data"])

### Salvando e Carregando a Tabela Delta

In [ ]:
# Especificando o caminho para o diretório Delta Lake onde os dados serão armazenados
delta_path = "/path/despachantes"

# Salvando o DataFrame original no formato Delta Lake
despachantes_df.write.format("delta").mode("overwrite").save(delta_path)

# Carregando o DeltaTable a partir do caminho especificado
delta_table = DeltaTable.forPath(spark, delta_path)

Salva o DataFrame original no formato Delta Lake, permitindo funcionalidades transacionais e a capacidade de realizar merges. Em seguida, carregamos a tabela Delta para permitir operações de atualização.

### Execução do Merge (Upsert)

In [ ]:
# Definindo a condição de merge (como exemplo, vamos usar a coluna "id")
condition = "target.id = source.id"

# Executando o merge/upsert
delta_table.alias("target") \
    .merge(novo_despachante.alias("source"), condition) \
    .whenMatchedUpdate(set={"nome": "source.nome", "status": "source.status", "cidade": "source.cidade",
                            "vendas": "source.vendas", "data": "source.data"}) \
    .whenNotMatchedInsert(values={"id": "source.id", "nome": "source.nome", "status": "source.status",
                                  "cidade": "source.cidade", "vendas": "source.vendas", "data": "source.data"}) \
    .execute()

Utiliza a operação merge para atualizar ou inserir dados na tabela Delta com base na condição especificada. Se um registro com o mesmo id já existir, ele é atualizado; caso contrário, um novo registro é inserido. Esta operação é crucial para manter a integridade dos dados sem duplicar informações.

### Verificação dos Dados Após Merge

In [ ]:
# Lendo o DataFrame resultante após o merge/upsert
despachantes_atualizados_df = spark.read.format("delta").load(delta_path)

# Exibindo o DataFrame resultante
despachantes_atualizados_df.orderBy("id").show()

Carrega e exibe os dados da tabela Delta para verificar se o merge foi realizado corretamente, mostrando os dados ordenados por id.

### Limpeza de Dados

In [ ]:
dbutils.fs.rm("/path/despachantes", recurse=True)

Remove o diretório e todos os arquivos associados